Create environment

In [ ]:

%cd /content
!git clone --depth 1 --filter=blob:none --sparse https://github.com/KyleLopin/MLClass.git
%cd MLClass
!git sparse-checkout set utility_functions

!pip install python-docx

import sys
sys.path.append('/content/MLClass/utility_functions')
from utility_functions.get_data import check_loaded, GetData

check_loaded()

Explore the data

In [ ]:
# get fish data
import pandas as pd

df = pd.read_csv("hf://datasets/scikit-learn/Fish/Fish.csv")

Perform basic regression

In [ ]:
# @title Student Questions (run to start)
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import random
# local files from git repo
from create_documents import create_document

# Create a text input field for the student's name
student_name_input = widgets.Text(
    description="Name:",
    placeholder="Enter your name",
)

# Create a submit button
submit_button = widgets.Button(description="Submit")

# Create an output widget to display the new form
output = widgets.Output()


def create_table(data):
    """
    Creates a widget table from a list of lists.

    Args:
        data (list of lists): Table data where first row is the header.

    Returns:
        widgets.VBox: The table as a widget.
    """
    num_rows = len(data)
    num_cols = len(data[0])

    grid = widgets.GridspecLayout(num_rows, num_cols)

    for row in range(num_rows):
        for col in range(num_cols):
            grid[row, col] = widgets.Label(value=str(data[row][col]))

    return grid


def ask_question(text, answer, table_data=None):
    """
    Displays a question with an optional table, accepts user input,
    and validates their answer with a margin of error.

    Args:
        text (str): The question to ask.
        answer (float): The correct answer.
        table_data (list of lists, optional): A table to display before the input.
    """
    output = widgets.Output()
    feedback_output = widgets.Output()  # Separate container for the feedback

    with output:
        clear_output(wait=True)

        # Display the question
        question_label = widgets.HTML(f"<h3>{text}</h3>")
        # display(question_label)

        # Display table if provided
        if table_data:
            table_widget = pd.DataFrame(table_data)  # ✅ Call reusable function
            display(table_widget)

        # Answer input field and submit button
        answer_input = widgets.FloatText(description="Your Answer:")
        submit_button = widgets.Button(description="Submit Answer")

        # Answer checking function
        def check_answer(b):
            student_answer = answer_input.value
            with feedback_output:
                print("next step")
                # clear_output(wait=True)
                # Clear the last response
                feedback_output.clear_output(wait=True)


                # Allow small floating-point error margin
                if abs(student_answer - answer) < 0.1:
                    display(widgets.HTML("<p style='color: green;'>✅ Correct!</p>"))
                else:
                    display(widgets.HTML(f"<p style='color: red;'>❌ Incorrect.</p>"))

    submit_button.on_click(check_answer)

    # Display input field and button
    container = [question_label, answer_input, submit_button, feedback_output]
    # display(widgets.VBox(container))
    return container

    # display(output)
    # display(feedback_output)


# Function to handle name submission and generate personalized data
def on_submit(b):
    all_containers = []
    with output:
        clear_output(wait=True)  # Clear previous forms

        # Get the student's name
        student_name = student_name_input.value.strip()

        if not student_name:
            print("❌ Please enter your name before proceeding.")
            return

        # Generate a unique seed for the student
        salt = student_name + "purple problem horse"
        seed = sum(ord(char) for char in salt)

        # Initialize GetData
        getdata = GetData(seed)

        # make questions
        # Generate data for the student
        species1, lengths, price, cost = getdata.load_data("fish cost")
        species2, weight_coefficient = getdata.load_data("fish coeff")

        weight_table = {"Fish number": ["Fish 1", "Fish 2", "Fish 3"],
                        "Length (cm)": lengths}
        doc_table = {"{Table 1}": [weight_table, ""]}
        questions = [(f"For the fish species {species2}.\n"
                       "\nWhat is the average increase in weight for a "
                       "1 cm increase in its length: ", weight_coefficient),
                      (f"What is the total cost for all of the {species1} listed below, "
                       f"if the fish cost {price} baht per 100 grams",
                       cost, weight_table)]

        # make document
        doc_table = {"{Table 1}": [pd.DataFrame(weight_table), ""]}
        create_document(student_name, "fish_assignment.docx",
                        "fish_regression_intro",
                        problems = [questions[0][0], questions[1][0]],
                        tables=doc_table)

        for question in questions:
            new_containers = ask_question(*question)
            all_containers.extend(new_containers)
        display(widgets.VBox(all_containers))


# Attach the function to the submit button
submit_button.on_click(on_submit)

# Display the student name form
display(student_name_input, submit_button, output)

Student work space for solving questions